In [1]:
import pandas as pd

In [2]:
# Parse data as pandas dataframes:
ratings = pd.read_csv('./BX-CSV-Dump/BX-Book-Ratings.csv', delimiter=';', header=0, encoding='latin1')
books = pd.read_csv('./BX-CSV-Dump/BX-Books.csv', delimiter=';', header=0, encoding='latin1', usecols=[0])
users = pd.read_csv('./BX-CSV-Dump/BX-Users.csv', delimiter=';', header=0, encoding='latin1', usecols=[0])

In [3]:
# Get the books with 200 ratings or more:
isbn_gte_200 = set(ratings.ISBN.value_counts()[ratings.ISBN.value_counts() >= 200].index)
len(isbn_gte_200)

195

In [4]:
# Get the books with 100 ratings or more:
isbn_gte_100 = set(ratings.ISBN.value_counts()[ratings.ISBN.value_counts() >= 100].index)
len(isbn_gte_100)

731

In [6]:
# Get the users that have rated at least 5 books in the `isbn_gte_200` set:
users_gte_5 = ratings[ratings.ISBN.map(lambda isbn: isbn in isbn_gte_200)]['User-ID'].value_counts()
users_gte_5 = set(users_gte_5[users_gte_5 >= 5].index)
len(users_gte_5)

2702

In [15]:
# Get the ratings of books in `isbn_gte_100` by users in `users_gte_5`:
reviews_to_insert = ratings[(
    ratings['User-ID'].map(lambda id: id in users_gte_5) & ratings.ISBN.map(lambda isbn: isbn in isbn_gte_100)
)].sort_values('User-ID')

len(reviews_to_insert)

80871

In [42]:
# Rename columns
reviews_to_insert.rename(columns={'User-ID': 'user_id', 'ISBN': 'isbn_10', 'Book-Rating': 'rating'}, inplace=True)

In [43]:
reviews_to_insert.to_csv('./reviews_to_insert.csv', index=False)

In [21]:
import requests

In [22]:
res = requests.get('https://randomuser.me/api/')

In [32]:
res.json()['results'][0]['login']['username']

'bluecat473'

In [39]:
reviews_to_insert.itertuples?

Signature: reviews_to_insert.itertuples(index: 'bool' = True, name: 'Optional[str]' = 'Pandas')
Docstring:
Iterate over DataFrame rows as namedtuples.

Parameters
----------
index : bool, default True
    If True, return the index as the first element of the tuple.
name : str or None, default "Pandas"
    The name of the returned namedtuples or None to return regular
    tuples.

Returns
-------
iterator
    An object to iterate over namedtuples for each row in the
    DataFrame with the first field possibly being the index and
    following fields being the column values.

See Also
--------
DataFrame.iterrows : Iterate over DataFrame rows as (index, Series)
    pairs.
DataFrame.items : Iterate over (column name, Series) pairs.

Notes
-----
The column names will be renamed to positional names if they are
invalid Python identifiers, repeated, or start with an underscore.
On python versions < 3.7 regular tuples are returned for DataFrames
with a large number of columns (>254).

Examples
